# Creating Formulaic Alphas
1. Series from yahoo finance
1. Series from investingdotcom

### Yahoo Finance

In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_ta as ta

import numpy as np
import pandas as pd
from numpy import abs
from numpy import log
from numpy import sign
from scipy.stats import rankdata

import lazypredict

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


In [2]:
tickers_df = pd.read_csv('../data/security_map_final.csv')
tickers_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,CIK,wiki_ticker,new_ticker
0,A,Agilent Technologies,Health Care,Health Care Equipment,1090872.0,A,A
1,AA,Alcoa Inc,NaN,NaN,NaN,AA,AA
2,AAL,American Airlines Group,Industrials,Airlines,6201.0,AAL,AAL
3,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,1158449.0,AAP,AAP
4,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",320193.0,AAPL,AAPL
...,...,...,...,...,...,...,...
740,YUM,Yum! Brands,Consumer Discretionary,Restaurants,1041061.0,YUM,YUM
741,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,1136869.0,ZBH,ZBH
742,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,877212.0,ZBRA,ZBRA
743,ZION,Zions Bancorporation,Financials,Regional Banks,109380.0,ZION,ZION


In [3]:
def calculate_5d_vwap(df):
    df = df.copy()
    df["5D Volume"] = df["Volume"].rolling(5).sum()
    df["Daily Avg Price"] = (df["Open"] + df["High"] + df["Low"])/3
    df["5D Volume Adj Typical Price"] = df["5D Volume"] * df["Daily Avg Price"]
    df["5D-VWAP"] = df["5D Volume Adj Typical Price"].cumsum() / df["Volume"].cumsum()
    df.dropna(inplace=True)
    return df

In [4]:
def calculate_pct_change(df):
    df = df.copy()
    df["Pct Change"] = df["Close"].pct_change()
    return df

In [5]:
from formulaic_alpha_lib import calculate_alphas

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

market_data_df_map = {}
for ticker in tickers_df['new_ticker'].tolist():
    try:
        market_data_df = pd.read_csv(f"../data/market_data_yf/{ticker}.csv").drop(["Unnamed: 0"], axis=1)
        market_data_df = calculate_5d_vwap(market_data_df)
        market_data_df = calculate_pct_change(market_data_df)
        market_data_df_map[ticker] = market_data_df
        market_data_df = calculate_alphas(market_data_df)
        market_data_df.dropna(inplace=True)
        market_data_df_map[ticker] = market_data_df
    except Exception as e:
        print(f"Ticker {ticker} failed")

/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: Run

Ticker AGN failed


/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Ticker AKS failed
Ticker ALXN failed


/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: Run

...

/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Ticker XL failed
Ticker XLNX failed


/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Ticker AABA failed


/Users/jonathanbenghiat/.pyenv/versions/3.9.10/envs/phobos/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Failed because data is in investing.com folder

In [7]:
len(market_data_df_map)

653

In [9]:
market_data_df_map["AAPL"]["Pct Change"]

258     -0.043723
259      0.017142
260     -0.022471
261      0.005747
262     -0.045711
           ...   
10626   -0.017929
10627    0.019245
10628   -0.017653
10629   -0.006912
10630    0.000928
Name: Pct Change, Length: 9657, dtype: float64

In [17]:
for ticker, df in market_data_df_map.items():
    df.to_csv(f"../data/alpha_augmented_market_data/{ticker}.csv")

### Investing dot com
### This data is very messy (because it is delisted tickers), have to handle differently

In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
# import pandas_ta as ta

import numpy as np
import pandas as pd
from numpy import abs
from numpy import log
from numpy import sign
from scipy.stats import rankdata

# import lazypredict

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV


In [2]:
tickers_df = pd.read_csv('../data/security_map_final.csv')
tickers_df

,Symbol,Security,GICS Sector,GICS Sub-Industry,CIK,wiki_ticker,new_ticker
0,A,Agilent Technologies,Health Care,Health Care Equipment,1090872.0,A,A
1,AA,Alcoa Inc,NaN,NaN,NaN,AA,AA
2,AAL,American Airlines Group,Industrials,Airlines,6201.0,AAL,AAL
3,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,1158449.0,AAP,AAP
4,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals",320193.0,AAPL,AAPL
...,...,...,...,...,...,...,...
738,YUM,Yum! Brands,Consumer Discretionary,Restaurants,1041061.0,YUM,YUM
739,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,1136869.0,ZBH,ZBH
740,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,877212.0,ZBRA,ZBRA
741,ZION,Zions Bancorporation,Financials,Regional Banks,109380.0,ZION,ZION


In [5]:
import os

In [90]:
from formulaic_alpha_lib import calculate_alphas
pd.options.mode.chained_assignment = None
import os


import warnings

def convert_volume(text):
    # Volume in millions or thousands
    if "M" in text:
        text = text[:-1]
        number = float(text) * 1000000
        
    elif "K" in text:
        text = text[:-1]
        number = float(text) * 1000
        
    else:
        raise ValueError
        
    return int(number)

market_data_df_map = {}
data_not_found = []
errors = []
for ticker in tickers_df['new_ticker'].tolist():
    if os.path.isfile(f"../data/market_data_yf/{ticker}.csv"):
        # We already processed it
        continue
    elif os.path.isfile(f"../data/market_data_investingdotcom/{ticker} Historical Data.csv"):
        try:
            print(ticker + " from investingdocom")
            market_data_df = pd.read_csv(f"../data/market_data_investingdotcom/{ticker} Historical Data.csv")
            market_data_df.rename(columns={"Price":"Close",
                  "Vol.":"Volume"}, inplace=True)
            market_data_df.Date = pd.to_datetime(market_data_df.Date, format="%m/%d/%Y")

            # We have to forward fill; missing data
            all_bdates = pd.DataFrame(pd.bdate_range(start=market_data_df.Date.min(), end=market_data_df.Date.max()), columns=["Date"])
            market_data_df = all_bdates.merge(market_data_df, how="left")
            market_data_df = market_data_df.ffill(limit=10)

            for col in ["Close", "Open", "High", "Low"]:
                # Now we have to interpolate between
                market_data_df[col] = market_data_df[col].interpolate()
                
            # Clean up volume
            market_data_df.Volume = market_data_df.Volume.apply(lambda x: convert_volume(x))

            market_data_df = calculate_5d_vwap(market_data_df)
            market_data_df = calculate_pct_change(market_data_df)
            market_data_df_map[ticker] = market_data_df
            market_data_df = calculate_alphas(market_data_df)
            market_data_df.dropna(inplace=True)
            market_data_df_map[ticker] = market_data_df
        except Exception as e:
            print(e)
            errors.append(ticker)
    else:
        print("#######################################################")
        print(ticker + " not found")
        data_not_found.append(ticker)
    

AGN from investingdocom
AKS from investingdocom
ALXN from investingdocom
APC from investingdocom
APOL from investingdocom
argument of type 'float' is not iterable
ARG from investingdocom
AVP from investingdocom
BCR from investingdocom
BF.B from investingdocom
BRK.B from investingdocom
BXLT from investingdocom
CAM from investingdocom
CELG from investingdocom
CERN from investingdocom
CFN from investingdocom
CPGX from investingdocom
CVC from investingdocom
CXO from investingdocom
DF from investingdocom
DEN from investingdocom
DRE from investingdocom
DTV from investingdocom
argument of type 'float' is not iterable
KODK from investingdocom
ENDPQ from investingdocom
VALPQ from investingdocom
argument of type 'float' is not iterable
ETFC from investingdocom
FDO from investingdocom
argument of type 'float' is not iterable
FHI from investingdocom
FLIR from investingdocom
BODY from investingdocom
argument of type 'float' is not iterable
FYBR from investingdocom
SNY from investingdocom
BPYU from 

### The errors are actually due to volume, we forward fill if below 10% is missing

In [101]:
for ticker in errors:
    
    print(ticker + " from investingdocom")
    market_data_df = pd.read_csv(f"../data/market_data_investingdotcom/{ticker} Historical Data.csv")
    market_data_df.rename(columns={"Price":"Close",
          "Vol.":"Volume"}, inplace=True)
    market_data_df.Date = pd.to_datetime(market_data_df.Date, format="%m/%d/%Y")

    all_bdates = pd.DataFrame(pd.bdate_range(start=market_data_df.Date.min(), end=market_data_df.Date.max()), columns=["Date"])
    market_data_df = all_bdates.merge(market_data_df, how="left")
    market_data_df = market_data_df.ffill(limit=10)

    for col in ["Close", "Open", "High", "Low"]:
        market_data_df[col] = market_data_df[col].interpolate()
        
    
    # If below 10% missing, we forward fill
    # If not, we just remove it entirely
    if market_data_df.Volume.isna().sum() / len(market_data_df.Volume) < 0.10:
        market_data_df.Volume = market_data_df.Volume.ffill()
    else:
        print(ticker)
        print(market_data_df.Volume.isna().sum() / len(market_data_df.Volume))
        continue
    
    market_data_df.Volume = market_data_df.Volume.apply(lambda x: convert_volume(x))

    market_data_df = calculate_5d_vwap(market_data_df)
    market_data_df = calculate_pct_change(market_data_df)
    market_data_df = calculate_alphas(market_data_df)
    market_data_df.dropna(inplace=True)
    market_data_df_map[ticker] = market_data_df
    

APOL from investingdocom
APOL
0.5243757431629013
DTV from investingdocom
VALPQ from investingdocom
FDO from investingdocom
BODY from investingdocom
BODY
0.5026642984014209
HCBK from investingdocom
HSP from investingdocom
ALPN from investingdocom
ALPN
0.43527416619559073
PGNPQ from investingdocom
PGNPQ
0.3570057581573896
SIAL from investingdocom
SWY from investingdocom
SWY
0.13740458015267176


Data is very dirty for APOL, BODY, ALPN, PGNPQ, SWY. We will exclude them.

In [102]:
len(market_data_df_map)

78

In [103]:
for ticker, df in market_data_df_map.items():
    df.to_csv(f"../data/alpha_augmented_market_data/{ticker}.csv")